Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))  
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  
  # Model.
  def model(data):
    print("layer1_weights: "+str(layer1_weights.shape))
    print("layer1_biases: "+str(layer1_biases.shape))
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    print("conv: "+str(conv.shape))
    hidden = tf.nn.relu(conv + layer1_biases)
    print("hidden: "+str(hidden.shape))
    print("-")
    
    
    print("layer2_weights: "+str(layer2_weights.shape))
    print("layer2_biases: "+str(layer2_biases.shape))
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    print("conv: "+str(conv.shape))
    hidden = tf.nn.relu(conv + layer2_biases)
    print("hidden: "+str(hidden.shape))
    print("-")
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    
    print("layer3_weights: "+str(layer3_weights.shape))
    print("layer3_biases: "+str(layer3_biases.shape))
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print("hidden: "+str(hidden.shape))
    print("-")

    print("layer4_weights: "+str(layer4_weights.shape))
    print("layer4_biases: "+str(layer4_biases.shape))
    result = tf.matmul(hidden, layer4_weights) + layer4_biases
    print("result: "+str(layer4_biases.shape))
    print("-")
    
    return result
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

layer1_weights: (5, 5, 1, 16)
layer1_biases: (16,)
conv: (16, 14, 14, 16)
hidden: (16, 14, 14, 16)
-
layer2_weights: (5, 5, 16, 16)
layer2_biases: (16,)
conv: (16, 7, 7, 16)
hidden: (16, 7, 7, 16)
-
layer3_weights: (784, 64)
layer3_biases: (64,)
hidden: (16, 64)
-
layer4_weights: (64, 10)
layer4_biases: (10,)
result: (10,)
-
layer1_weights: (5, 5, 1, 16)
layer1_biases: (16,)
conv: (10000, 14, 14, 16)
hidden: (10000, 14, 14, 16)
-
layer2_weights: (5, 5, 16, 16)
layer2_biases: (16,)
conv: (10000, 7, 7, 16)
hidden: (10000, 7, 7, 16)
-
layer3_weights: (784, 64)
layer3_biases: (64,)
hidden: (10000, 64)
-
layer4_weights: (64, 10)
layer4_biases: (10,)
result: (10,)
-
layer1_weights: (5, 5, 1, 16)
layer1_biases: (16,)
conv: (10000, 14, 14, 16)
hidden: (10000, 14, 14, 16)
-
layer2_weights: (5, 5, 16, 16)
layer2_biases: (16,)
conv: (10000, 7, 7, 16)
hidden: (10000, 7, 7, 16)
-
layer3_weights: (784, 64)
layer3_biases: (64,)
hidden: (10000, 64)
-
layer4_weights: (64, 10)
layer4_biases: (10,)
resul

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.725944
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.806348
Minibatch accuracy: 43.8%
Validation accuracy: 42.6%
Minibatch loss at step 100: 0.738228
Minibatch accuracy: 81.2%
Validation accuracy: 68.7%
Minibatch loss at step 150: 0.859987
Minibatch accuracy: 75.0%
Validation accuracy: 73.2%
Minibatch loss at step 200: 1.188212
Minibatch accuracy: 75.0%
Validation accuracy: 69.0%
Minibatch loss at step 250: 1.414016
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 300: 0.801751
Minibatch accuracy: 68.8%
Validation accuracy: 79.4%
Minibatch loss at step 350: 0.813376
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.287928
Minibatch accuracy: 93.8%
Validation accuracy: 78.5%
Minibatch loss at step 450: 0.464904
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 500: 0.400863
Minibatch accuracy: 93.8%
Validation accuracy: 81.3%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [39]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))  
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  
  # Model.
  def model(data):
    print("layer1_weights: "+str(layer1_weights.shape))
    print("layer1_biases: "+str(layer1_biases.shape))
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    print("conv: "+str(conv.shape))
    hidden = tf.nn.relu(conv + layer1_biases)
    print("hidden: "+str(hidden.shape))
    ksize = [1, 2, 2, 1]
    strides = [1,2,2,1]
    hidden = tf.nn.max_pool(hidden, ksize, strides, padding='SAME')
    print("hidden2: "+str(hidden.shape))
    print("-")
    
    
    print("layer2_weights: "+str(layer2_weights.shape))
    print("layer2_biases: "+str(layer2_biases.shape))
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    print("conv: "+str(conv.shape))
    hidden = tf.nn.relu(conv + layer2_biases)
    print("hidden: "+str(hidden.shape))
    ksize = [1, 2, 2, 1]
    strides = [1,2,2,1]
    hidden = tf.nn.max_pool(hidden, ksize, strides, padding='SAME')
    print("hidden2: "+str(hidden.shape))
    print("-")
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    
    print("layer3_weights: "+str(layer3_weights.shape))
    print("layer3_biases: "+str(layer3_biases.shape))
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print("hidden: "+str(hidden.shape))
    print("-")

    print("layer4_weights: "+str(layer4_weights.shape))
    print("layer4_biases: "+str(layer4_biases.shape))
    result = tf.matmul(hidden, layer4_weights) + layer4_biases
    print("result: "+str(layer4_biases.shape))
    print("-")
    
    return result
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

layer1_weights: (5, 5, 1, 16)
layer1_biases: (16,)
conv: (16, 28, 28, 16)
hidden: (16, 28, 28, 16)
hidden2: (16, 14, 14, 16)
-
layer2_weights: (5, 5, 16, 16)
layer2_biases: (16,)
conv: (16, 14, 14, 16)
hidden: (16, 14, 14, 16)
hidden2: (16, 7, 7, 16)
-
layer3_weights: (784, 64)
layer3_biases: (64,)
hidden: (16, 64)
-
layer4_weights: (64, 10)
layer4_biases: (10,)
result: (10,)
-
layer1_weights: (5, 5, 1, 16)
layer1_biases: (16,)
conv: (10000, 28, 28, 16)
hidden: (10000, 28, 28, 16)
hidden2: (10000, 14, 14, 16)
-
layer2_weights: (5, 5, 16, 16)
layer2_biases: (16,)
conv: (10000, 14, 14, 16)
hidden: (10000, 14, 14, 16)
hidden2: (10000, 7, 7, 16)
-
layer3_weights: (784, 64)
layer3_biases: (64,)
hidden: (10000, 64)
-
layer4_weights: (64, 10)
layer4_biases: (10,)
result: (10,)
-
layer1_weights: (5, 5, 1, 16)
layer1_biases: (16,)
conv: (10000, 28, 28, 16)
hidden: (10000, 28, 28, 16)
hidden2: (10000, 14, 14, 16)
-
layer2_weights: (5, 5, 16, 16)
layer2_biases: (16,)
conv: (10000, 14, 14, 16)
hid

In [40]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.820326
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.151432
Minibatch accuracy: 31.2%
Validation accuracy: 25.6%
Minibatch loss at step 100: 0.905109
Minibatch accuracy: 81.2%
Validation accuracy: 63.3%
Minibatch loss at step 150: 0.937130
Minibatch accuracy: 62.5%
Validation accuracy: 69.1%
Minibatch loss at step 200: 1.830042
Minibatch accuracy: 68.8%
Validation accuracy: 50.4%
Minibatch loss at step 250: 1.569284
Minibatch accuracy: 56.2%
Validation accuracy: 73.2%
Minibatch loss at step 300: 0.785148
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.685983
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.268304
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.356184
Minibatch accuracy: 93.8%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.383192
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---